## Late merge with soft parameters ###

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
import random
from sklearn.metrics import accuracy_score

In [2]:
class hard_dataloader(Dataset):
    def __init__(self, data_dir , train):
        self.data_dir = data_dir
        self.all_classes = ['NotSnow','Snow']
        self.class_indexes = {cls: idx for idx, cls in enumerate(self.all_classes)}
        self.dataset = self.dataset_maker(train)
        self.length = len(self.dataset)
    def dataset_maker(self, train): # this will compile all the folder paths and the corresponding classes in a tuple to be stored in the class
        data_list = []
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.data_dir, class_name)
            for fold in os.listdir(all_class_folders):
                if fold == ".DS_Store":
                    continue
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        random.shuffle(data_list)
        if train:
            return data_list[:int(0.8*len(data_list))]
        else:
            return data_list[int(0.8*len(data_list)):]
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample_path, sample_class = self.dataset[index]
        pic1 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png')).convert('L')) # image 1 in tensor
        pic2 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '1.png')).convert('L')) # image 2 in tensor
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
        ])
        pic1 = transform(pic1)
        pic2 = transform(pic2)
        return pic1.unsqueeze(0) , pic2.unsqueeze(0), sample_class

In [3]:
e_soft_training = hard_dataloader("./dataset/", True)
e_soft_testing = hard_dataloader("./dataset/", False)

In [4]:
print(len(e_soft_testing))
print(len(e_soft_training))

714
2852


In [5]:
sample3 = e_soft_training[242]
print(sample3[0].shape)
print(sample3[1].shape)
print(sample3[2])

torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
0


In [6]:
class late_soft_cnn(nn.Module):
    def __init__(self):
        super(late_soft_cnn, self).__init__()
        self.conv1 = nn. Conv2d(1, 4, kernel_size=2, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn. Conv2d(4, 6, kernel_size=4, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=5, stride=5)
        # self.conv4 = nn. Conv2d(4, 6, kernel_size=3, stride=1)
        # self.pool4 = nn.MaxPool2d(kernel_size=5, stride=1)
        self.fc1 = nn.Linear (588, 2)
        
    def forward (self, x1,x2):
        x1 = F.relu(self.conv1(x1))
        x1 = self.pool(x1)
        x2 = F.relu(self.conv1(x2))
        x2 = self.pool(x2)
        x1 = F.relu(self.conv2(x1))
        x1 = self.pool2(x1)
        x2 = F.relu(self.conv2(x2))
        x2 = self.pool2(x2)
        # x1 = F.relu(self.conv4(x1))
        # x2 = F.relu(self.conv4(x2))
        # x1 = self.pool4(x1)
        # x2 = self.pool4(x2)
        x= torch.cat((x1, x2), dim=1)
        x=torch.flatten (x, 1)
        x = self.fc1(x)
        return x

In [7]:
# hyper parameters
learning_rate_4= 0.001
epochs_4 = 5

In [8]:
# training the neural net with the stacking dataset
model4 = late_soft_cnn()
criterion4 = nn.CrossEntropyLoss()
optimiser4 = optim.Adam(model4.parameters(), lr = learning_rate_4)

In [9]:
# model params
total_params = sum(p.numel() for p in model4.parameters())
print(f'the total number of parameters in this model are {total_params}')

the total number of parameters in this model are 1588


In [10]:
model4.train()
for epoch in range(epochs_4):
    running_loss = 0 
    labels_list = []
    predicted_list = []
    print(f'this is epoch number {epoch}')
    for i, (input1,input2, label) in enumerate(e_soft_training):
        optimiser4.zero_grad()
        labels_list.append(label)
        label = torch.tensor(label).view(-1)
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model4(input1, input2)
        loss = criterion4(outputs, label)
        loss.backward()
        optimiser4.step()
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.append(predicted.item())
        running_loss += loss.item()
        if i % 1000 == 999:   
            print(' loss: %.3f accuracy: %.3f' %(running_loss / 10,  100 *accuracy_score(labels_list, predicted_list)))
            running_loss = 0
          

this is epoch number 0
this is the 0 iteration
 loss: 37.114 accuracy: 82.400
this is the 1000 iteration
 loss: 26.182 accuracy: 85.900
this is the 2000 iteration
this is epoch number 1
this is the 0 iteration
 loss: 22.568 accuracy: 92.000
this is the 1000 iteration
 loss: 19.497 accuracy: 92.200
this is the 2000 iteration
this is epoch number 2
this is the 0 iteration
 loss: 17.292 accuracy: 93.700
this is the 1000 iteration
 loss: 15.931 accuracy: 93.600
this is the 2000 iteration
this is epoch number 3
this is the 0 iteration
 loss: 14.869 accuracy: 94.300
this is the 1000 iteration
 loss: 14.237 accuracy: 94.550
this is the 2000 iteration
this is epoch number 4
this is the 0 iteration
 loss: 13.448 accuracy: 95.000
this is the 1000 iteration
 loss: 13.176 accuracy: 95.400
this is the 2000 iteration


In [11]:
model4.eval()
loss = 0 
labels_list = []
predicted_list = []
with torch.no_grad():
    for i, (input1,input2, label) in enumerate(e_soft_testing):
        label = torch.tensor(label).view(-1)
        labels_list.append(label.item())
        outputs = model4(input1, input2)
        loss += criterion4(outputs, label).item()
        predicted = outputs.argmax(dim = 1 , keepdim = True)
        predicted_list.append(predicted.item())

In [12]:
print(f'the model accuracy is {accuracy_score(labels_list, predicted_list)}')

the model accuracy is 0.969187675070028
